In [16]:
import psycopg2
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count,regexp_replace,expr

spark = SparkSession.builder.getOrCreate()

In [2]:
spark

In [17]:

df = spark.read.csv("steam.csv", header=True, inferSchema=True)


In [18]:
#load data to postgre from df(csv file dataframe)
try:
    df.write.format("jdbc").options(

        url="jdbc:postgresql://localhost:5432/netflixShows",
        driver="org.postgresql.Driver",
        dbtable="game_info",
        user="postgres",
        password="12345"
        ).mode("overwrite").save()
    print("data loaded successfully")
except Exception as e:
    print("data load error "+str(e))

data loaded successfully


In [19]:
#extract data from postgre

postgres_url = "jdbc:postgresql://localhost:5432/netflixShows"
properties = {
    "user": "postgres",
    "password": "12345",
    "driver": "org.postgresql.Driver"
}

# Read data from PostgreSQL
new_df = spark.read.jdbc(url=postgres_url, table="game_info", properties=properties)

In [ ]:
#display df
new_df.show()

In [21]:
#transform
new_df = new_df.withColumn("english", when(new_df["english"] == 1, 'yes').otherwise(new_df["english"]))
new_df = new_df.withColumn("required_age", when(new_df["required_age"] == 0, None).otherwise(new_df["required_age"]))

In [ ]:
#check if there is any null values
null_counts = new_df.select([count(when(col(c).isNull(), c)).alias(c) for c in new_df.columns])

null_counts.show()

In [ ]:
#transform
for col_name in new_df.columns:
    new_df = new_df.withColumn(col_name, when(new_df[col_name].isNull(), 'default_value').otherwise(new_df[col_name]))
new_df = new_df.na.fill({'director':'default'})

In [22]:
#Load new df to MySql
new_df.write.format("jdbc").options(
    
    url="jdbc:mysql://localhost:3306/test",
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="game_info",
    user="root",
    password="berkberk09"
    ).mode("overwrite").save()

In [ ]:
#null count for each columns
null_counts = new_df.select([count(when(col(c).isNull(), c)).alias(c) for c in new_df.columns])

null_counts.show()